In [1]:
# importing necessary libraries

import numpy as np
import wbgapi as wb
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.optimize as opt
from scipy.stats import norm
%matplotlib inline
# creating a country codes list and indicator id list
# EN.ATM.CO2E.PC = CO2 emissions (metric tons per capita)

country_codes = ["GBR","USA","BGD"]
indicator_id = ["EN.ATM.CO2E.PC"]
# creating dataframe for clustering

df = wb.data.DataFrame(indicator_id,country_codes,skipAggs=True, mrv=10, columns='series', numericTimeKeys=True)
df.index.names = ['country','year']
df["co2"] = pd.to_numeric(df["EN.ATM.CO2E.PC"])
df = df.drop(['EN.ATM.CO2E.PC'], axis =1)
df

co2
country year           
BGD     2009   0.306665
        2010   0.342740
        2011   0.364567
        2012   0.384025
        2013   0.396566
        2014   0.412899
        2015   0.458798
        2016   0.466776
        2017   0.492907
        2018   0.512837
GBR     2009   7.468655
        2010   7.675926
        2011   7.025425
        2012   7.332471
        2013   7.064903
        2014   6.421598
        2015   6.148545
        2016   5.774742
        2017   5.546266
        2018   5.398708
USA     2009  16.818868
        2010  17.434196
        2011  16.599404
        2012  15.771145
        2013  16.102958
        2014  16.026379
        2015  15.535342
        2016  15.131747
        2017  14.805882
        2018  15.240875